In [9]:
import pandas as pd
VTH = pd.read_csv('VenmoStatement_Sep_2025.csv', header=2)
FOS = pd.read_csv('2025-2026 Registry and Budget - VenmoCodeF (5).csv')


In [21]:
filtered = VTH.iloc[1:]
paidNames = pd.DataFrame(columns=['Name', 'UCSD Email', "money"])

for index, sheetsEntry in FOS.iterrows():
    if sheetsEntry['UCSD Email'] is not None and not pd.isnull(sheetsEntry['UCSD Email']):
        if sheetsEntry['Paid?'] == 'Select' or sheetsEntry['Paid?'] == 'No':
            #Code to process non paid members
            for i, venmoEntry in filtered.iterrows():
                if pd.notnull(venmoEntry['Note']) and (sheetsEntry['UCSD Email'].lower() in venmoEntry['Note'].lower()) and ('@' in sheetsEntry['UCSD Email']) and (venmoEntry['Amount (total)'] == '+ $35.00'):
                    sheetsEntry['Paid?'] = 'Yes'
                    paidNames.loc[len(paidNames)] = [sheetsEntry['Name'], sheetsEntry['UCSD Email'], venmoEntry['Amount (total)']]
                    filtered = filtered.drop(i)  # Remove matched row (NOT TESTED)
                    break
        else: 
            #Add members who already paid to output
            paidNames.loc[len(paidNames)] = [sheetsEntry['Name'], sheetsEntry['UCSD Email'], '35']

FOS.to_csv('output.csv', index=False)
onlyBool = FOS['Paid?']
onlyBool.to_csv('outputYesNo.csv', index = False)

#Code to filter out the venmo transactions that were wrong
correct = pd.DataFrame(columns=['Name', 'Venmo Caption', "money"])
wrong = filtered.copy()
for index, venmoEntry in wrong.iterrows():
    for i, sheetsEntry in FOS.iterrows():
        if sheetsEntry['UCSD Email'] is not None and not pd.isnull(sheetsEntry['UCSD Email']):
            if pd.notnull(venmoEntry['Note']) and (sheetsEntry['UCSD Email'].lower() in venmoEntry['Note'].lower()) and ('@' in sheetsEntry['UCSD Email']) and (venmoEntry['Amount (total)'] == '+ $35.00'):
                correct.loc[len(correct)] = [venmoEntry['From'], venmoEntry['Note'], venmoEntry['Amount (total)']]
                wrong = wrong.drop(index)
                break

correct.to_csv('Venmo_Transac_Correct.csv')     
wrong.to_csv('Venmo_Transac_Wrong.csv')